In [2]:
%run imports_and_functions.ipynb

C:\Users\Jason\AppData\Local\conda\conda\envs\DSI-6\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Vectorize Functions of Business Categories for a Location 

In [3]:
df = pd.read_csv('LasVegasBusinessesFinal.csv',sep=';',compression='gzip',
                 index_col=['lat_index','long_index'])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 12882 entries, (36.058, -115.1820861) to (36.183, -115.179353601)
Data columns (total 18 columns):
lat_index.1      12882 non-null float64
long_index.1     12882 non-null float64
name             12882 non-null object
address          12470 non-null object
attributes       11131 non-null object
categories       12851 non-null object
city             12882 non-null object
latitude         12882 non-null float64
longitude        12882 non-null float64
neighborhood     10489 non-null object
postal_code      12882 non-null int64
review_count     12882 non-null int64
stars            12882 non-null float64
state            12882 non-null object
price            12882 non-null int64
main_category    12845 non-null object
distance         12882 non-null float64
alias            12882 non-null object
dtypes: float64(6), int64(3), object(9)
memory usage: 1.9+ MB


In [10]:
# df.drop(['lat_index.1.1', 'long_index.1.1'], axis=1, inplace=True)
# df.drop(['name.1'], axis=1, inplace=True)

In [5]:
def categories_to_alias(dfrow) :
    catstring = dfrow['categories']
    if type(catstring) != str : return str(dfrow['price']) + '_other'
    lis = list(set(catstring.split(', ')))
    lis = map(get_alias,lis)
    lis = set(map((lambda x : str(dfrow['price']) + x),lis))
    return ' '.join(lis)

In [6]:
def get_alias(cat) :
    try :
        alias = '_' + str(cat_alias_dict[map_category(cat)])
    except : alias = '_other'
    return alias

### Yelp Categories to Alias and Alias to Category Dictionary

In [27]:
cat_dict = json.load(open('Yelp_categoriesList.json','r'))
df = pd.DataFrame(data=cat_dict)
df.head()

,alias,country_blacklist,country_whitelist,parents,title
0,3dprinting,NaN,NaN,[localservices],3D Printing
1,abruzzese,NaN,[IT],[italian],Abruzzese
2,absinthebars,NaN,[CZ],[bars],Absinthe Bars
3,acaibowls,"[TR, PL, AR, MX, IT, CL]",NaN,[food],Acai Bowls
4,accessories,NaN,NaN,[fashion],Accessories


In [28]:
cat_alias_dict = {}
for index in df.index :
    cat_alias_dict[df.loc[index,'alias']] = df.loc[index,'title']
    cat_alias_dict[df.loc[index,'title']] = df.loc[index,'alias']
len(cat_alias_dict)

3130

In [29]:
json.dump(cat_alias_dict, open('cat_alias_dict.json','w'))

### Mapping to Only Top 150 Categories

In [36]:
cat_alias_dict = json.load(open('cat_alias_dict.json','r'))

In [7]:
## Function to map titles to macro-categories if not in top 150

dict_2A = json.load(open('dict_2A.json','r'))
dict_2B = json.load(open('dict_2B.json','r'))

def map_category(category):
    if category not in top_148:
        if category in dict_2B.keys():
            title = dict_2B[category]
        else:
            title = category
        if title in dict_2A.keys():
            title_2 = dict_2A[title]
            return(title_2)
        else:
            return(title)
    else:
        return(category)

In [176]:
map_category('martialarts')

'Active Life'

In [8]:
df['alias'] = df.apply(categories_to_alias, axis=1)

In [178]:
df['alias'].value_counts()

0_health 0_physicians                                                                                            260
0_homeservices                                                                                                   181
0_lawyers 0_professional                                                                                         181
0_theater 0_arts                                                                                                 175
0_hotelstravel                                                                                                   149
0_auto 0_autorepair                                                                                              135
0_homeservices 0_apartments 0_realestate                                                                         135
0_localservices                                                                                                  130
0_arts                                                          

In [12]:
df.to_csv('LasVegasBusinessesFinal.csv',sep=';',compression='gzip')

### Assembling Category Features Header List

In [97]:
list1 = list(map((lambda x : cat_alias_dict[x]), top_148))
X_features = ['landarea','waterarea']
for alias in list1 :
    for n in range(0,5) :
        for w in ['near','far'] :
            X_features.append(str(n) + '_' + alias + '_' + w)
X_features[:33]

['landarea',
 'waterarea',
 '0_restaurants_near',
 '0_restaurants_far',
 '1_restaurants_near',
 '1_restaurants_far',
 '2_restaurants_near',
 '2_restaurants_far',
 '3_restaurants_near',
 '3_restaurants_far',
 '4_restaurants_near',
 '4_restaurants_far',
 '0_shopping_near',
 '0_shopping_far',
 '1_shopping_near',
 '1_shopping_far',
 '2_shopping_near',
 '2_shopping_far',
 '3_shopping_near',
 '3_shopping_far',
 '4_shopping_near',
 '4_shopping_far',
 '0_food_near',
 '0_food_far',
 '1_food_near',
 '1_food_far',
 '2_food_near',
 '2_food_far',
 '3_food_near',
 '3_food_far',
 '4_food_near',
 '4_food_far',
 '0_beautysvc_near']

In [101]:
joblib.dump(X_features, 'X_features.joblib')

['X_features.joblib']

### Getting X_Train Lat/Longs of Businesses within 1000 meters

In [157]:
trainrowcoords = pd.read_csv('LasVegasTrainRowCoords.csv',sep=';',compression='gzip')
trainrowcoords = trainrowcoords.set_index('Unnamed: 0')
trainrowcoords.index.name = 'geoid'
trainrowcoords.head()

,distance,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea,block
geoid,,,,,,,,,,,
320030029581005,5403.219921,U,36.093640,-115.241871,320030029581,25757,1495,155,54924,0,1005
320030029572000,5808.613932,U,36.096091,-115.248927,320030029572,52256,884,437,728659,0,2000
320030010042004,3855.731439,U,36.155052,-115.198804,320030010042,25408,1249,32,17103,0,2004
320030010031016,4532.868829,U,36.154625,-115.220151,320030010031,24759,1058,29,41920,0,1016
320030029381020,3136.357472,U,36.137785,-115.219814,320030029381,34314,1539,35,39561,0,1020


In [158]:
trainrowcoords.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1784 entries, 320030029581005 to 5320030024061000
Data columns (total 11 columns):
distance         1784 non-null float64
urban            1784 non-null object
latitude         1784 non-null float64
longitude        1784 non-null float64
blockgroup       1784 non-null int64
income           1784 non-null int64
blockgrouppop    1784 non-null int64
population       1784 non-null int64
landarea         1784 non-null int64
waterarea        1784 non-null int64
block            1784 non-null int64
dtypes: float64(3), int64(7), object(1)
memory usage: 167.2+ KB


In [159]:
df['distance'] = df.apply(
    (lambda row : calcDist(row['latitude'],row['longitude'],lat, long)),         
    axis=1)

### Function Assembling Pooled Category Corpus for One Search Radius

In [194]:
def makecorpus(df, lat, long) :
    distances = calcDistancesV2(df, lat, long)
    distances = pd.DataFrame(data=distances, index=distances.index, columns=['distances'])
    distances = distances.join(df['alias']) #['alias'] = df.loc[distances.index, 'alias']
    print(distances.shape)
    temp = distances.apply(addproximity, axis=1)
    return ' '.join(temp)

def addproximity(dfrow) :
    w = ('near' if (dfrow.loc['distances'] <= 500) else 'far')
    temp = dfrow.loc['alias'].split(' ')
    temp = list(map((lambda x : (x + '_' + w)), temp))
    return ' '.join(temp)

def calcDistancesV2(multiindexdf, lat, long) :
    minlat = lat - lat1000m
    maxlat = lat + lat1000m
    minlong = long - lat1000m
    maxlong = long + lat1000m
#     multiindexdf = multiindexdf.sort_index(level = [0,1])
    idx = pd.IndexSlice
    df = multiindexdf.loc[idx[minlat:maxlat, minlong:maxlong], ['latitude','longitude']]
    distances = pd.Series(index=df.index)
    for index in df.index :
        distances[index] = calcDist(lat, long, df.loc[index,'latitude'].values[0], 
                                    df.loc[index,'longitude'].values[0])
    distances = distances[distances<1000]
    return distances

In [78]:
lat, long = 36.116803,-115.161605
lat1000m = -115.161605-(-115.172713)
calcDist(lat, long, 36.116803, -115.172713)

1000.0592673150582

In [149]:
%%timeit
test2 = calcDistancesV2(df, lat, long)

2.88 s ± 164 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [148]:
test2

lat_index  long_index 
36.109     -115.165950    964.483037
           -115.163254    925.929993
           -115.156894    918.772883
           -115.156894    918.772883
36.110     -115.169123    985.679881
           -115.168817    994.135278
           -115.168745    995.764181
           -115.168740    949.649009
           -115.168633    970.928854
           -115.168140    961.279293
           -115.168137    960.950512
           -115.155337    918.939307
           -115.155311    924.265941
           -115.155311    924.265941
           -115.154282    964.107418
36.111     -115.169640    970.106144
           -115.169562    999.226337
           -115.169470    990.709754
           -115.169417    986.092233
           -115.168884    953.667840
           -115.168838    939.250340
           -115.168838    939.250340
           -115.168835    936.078046
           -115.168129    866.507075
           -115.164153    655.677601
36.112     -115.171174    999.045499
           -115

In [195]:
makecorpus(df, lat, long)

(2773, 2)


'0_health_far 0_physicians_far 0_medcenters_far 0_nightlife_far 3_beautysvc_far 3_massage_far 3_skincare_far 2_restaurants_far 2_newamerican_far 3_beautysvc_far 3_massage_far 3_skincare_far 2_restaurants_far 2_newamerican_far 2_shopping_far 2_burgers_far 2_catering_far 2_restaurants_far 2_tradamerican_far 2_newamerican_far 2_eventservices_far 2_sandwiches_far 2_burgers_far 2_restaurants_far 2_bars_far 2_cocktailbars_far 2_nightlife_far 2_beautysvc_far 2_cosmetics_far 2_shopping_far 2_asianfusion_far 2_restaurants_far 2_japanese_far 3_menscloth_far 3_accessories_far 3_fashion_far 3_shopping_far 3_womenscloth_far 1_restaurants_far 1_pizza_far 1_italian_far 3_nightlife_far 3_arts_far 3_active_far 3_bars_far 0_arts_far 0_localflavor_far 3_nightlife_far 3_arts_far 3_active_far 3_bars_far 0_arts_far 0_localflavor_far 1_restaurants_far 1_cafes_far 1_food_far 1_coffee_far 3_fashion_far 3_shopping_far 3_womenscloth_far 2_bars_far 2_restaurants_far 2_tradamerican_far 2_sportsbars_far 2_nightlife

### Extracting Pooled Category Corpus of all Businesses within Radius and Count Vectorizing Corpus

In [181]:
def make_X_train(coorddf, businessesdf) :
    for index in coorddf.index :
        coorddf.loc[index,'corpus'] = makecorpus(businessesdf, 
                                        coorddf.loc[index,'latitude'], coorddf.loc[index,'longitude'])

In [196]:
make_X_train(trainrowcoords, df)

(404, 2)
(323, 2)
(300, 2)
(260, 2)
(1250, 2)
(434, 2)
(1076, 2)
(524, 2)
(529, 2)
(334, 2)
(299, 2)
(537, 2)
(328, 2)
(133, 2)
(3045, 2)
(832, 2)
(359, 2)
(257, 2)
(301, 2)
(923, 2)
(350, 2)
(1465, 2)
(345, 2)
(714, 2)
(281, 2)
(1119, 2)
(322, 2)
(257, 2)
(220, 2)
(388, 2)
(572, 2)
(1469, 2)
(922, 2)
(2644, 2)
(222, 2)
(1235, 2)
(170, 2)
(246, 2)
(1349, 2)
(298, 2)
(268, 2)
(1692, 2)
(282, 2)
(1769, 2)
(95, 2)
(1506, 2)
(336, 2)
(3470, 2)
(745, 2)
(457, 2)
(172, 2)
(140, 2)
(458, 2)
(2651, 2)
(3677, 2)
(423, 2)
(132, 2)
(196, 2)
(271, 2)
(532, 2)
(145, 2)
(583, 2)
(493, 2)
(523, 2)
(1367, 2)
(673, 2)
(717, 2)
(418, 2)
(289, 2)
(579, 2)
(1701, 2)
(1265, 2)
(223, 2)
(776, 2)
(704, 2)
(590, 2)
(406, 2)
(311, 2)
(319, 2)
(1329, 2)
(1809, 2)
(144, 2)
(897, 2)
(436, 2)
(505, 2)
(440, 2)
(311, 2)
(790, 2)
(224, 2)
(770, 2)
(174, 2)
(553, 2)
(813, 2)
(484, 2)
(670, 2)
(388, 2)
(3348, 2)
(1458, 2)
(273, 2)
(2544, 2)
(1622, 2)
(419, 2)
(1825, 2)
(1063, 2)
(1139, 2)
(219, 2)
(496, 2)
(555, 2)
(2

(241, 2)
(980, 2)
(363, 2)
(698, 2)
(317, 2)
(1464, 2)
(739, 2)
(289, 2)
(289, 2)
(466, 2)
(375, 2)
(1216, 2)
(562, 2)
(218, 2)
(1169, 2)
(1348, 2)
(567, 2)
(499, 2)
(692, 2)
(224, 2)
(220, 2)
(341, 2)
(740, 2)
(574, 2)
(608, 2)
(148, 2)
(1938, 2)
(400, 2)
(130, 2)
(541, 2)
(172, 2)
(197, 2)
(223, 2)
(168, 2)
(2393, 2)
(238, 2)
(133, 2)
(374, 2)
(1251, 2)
(1348, 2)
(240, 2)
(249, 2)
(386, 2)
(352, 2)
(2304, 2)
(1435, 2)
(671, 2)
(167, 2)
(317, 2)
(1372, 2)
(969, 2)
(273, 2)
(200, 2)
(223, 2)
(740, 2)
(1294, 2)
(1549, 2)
(459, 2)
(168, 2)
(708, 2)
(373, 2)
(1038, 2)
(197, 2)
(1510, 2)
(1945, 2)
(1036, 2)
(287, 2)
(648, 2)
(277, 2)
(788, 2)
(630, 2)
(663, 2)
(417, 2)
(125, 2)
(2257, 2)
(310, 2)
(432, 2)
(727, 2)
(436, 2)
(2716, 2)
(236, 2)
(382, 2)
(201, 2)
(119, 2)
(1508, 2)
(308, 2)
(715, 2)
(587, 2)
(527, 2)
(505, 2)
(365, 2)
(363, 2)
(984, 2)
(1367, 2)
(1607, 2)
(148, 2)
(3082, 2)
(492, 2)
(1482, 2)
(568, 2)
(653, 2)
(2377, 2)
(1318, 2)
(720, 2)
(242, 2)
(1243, 2)
(2566, 2)
(135, 2)


(1002, 2)
(3296, 2)
(3270, 2)
(1318, 2)
(471, 2)
(1834, 2)
(2240, 2)
(2240, 2)
(2240, 2)
(2240, 2)
(2600, 2)
(421, 2)
(421, 2)
(421, 2)
(421, 2)
(421, 2)
(934, 2)
(934, 2)
(934, 2)
(288, 2)
(288, 2)
(366, 2)
(366, 2)
(366, 2)
(2732, 2)
(2732, 2)
(2732, 2)
(2732, 2)
(2732, 2)


In [199]:
trainrowcoords.to_csv('CorpusLasVegasTrainRowCoords.csv',sep=';',compression='gzip')
trainrowcoords.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1784 entries, 320030029581005 to 5320030024061000
Data columns (total 12 columns):
distance         1784 non-null float64
urban            1784 non-null object
latitude         1784 non-null float64
longitude        1784 non-null float64
blockgroup       1784 non-null int64
income           1784 non-null int64
blockgrouppop    1784 non-null int64
population       1784 non-null int64
landarea         1784 non-null int64
waterarea        1784 non-null int64
block            1784 non-null int64
corpus           1784 non-null object
dtypes: float64(3), int64(7), object(2)
memory usage: 261.2+ KB


In [201]:
import scipy.sparse

X_features = joblib.load('X_features.joblib')

vect = CountVectorizer(dtype='uint16',min_df=1, vocabulary=X_features,
                       decode_error='ignore', lowercase=True) 

X_sparse = vect.transform(trainrowcoords['corpus'])

# scipy.sparse.save_npz('LasVegas_X_sparse.npz', X_sparse)

In [202]:
# X_sparse = scipy.sparse.load_npz('LasVegas_X_sparse.npz')
dfvect = pd.DataFrame(X_sparse.toarray(),dtype='uint16',columns=X_features)
dfvect.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1784 entries, 0 to 1783
Columns: 1482 entries, landarea to 4_physicaltherapy_far
dtypes: uint16(1482)
memory usage: 5.0 MB


In [203]:
dfvect.head()

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
0,0,0,0,1,8,37,4,26,0,0,0,0,0,4,0,27,4,6,2,0,0,0,0,0,4,19,3,7,0,0,0,0,0,2,3,32,5,16,0,0,0,0,2,14,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,31,0,22,0,0,0,0,0,3,0,17,0,5,0,2,0,0,0,0,0,7,0,7,0,0,0,0,0,2,0,30,0,8,0,0,0,0,1,7,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,3,3,22,3,16,0,0,0,0,4,12,4,17,10,38,1,6,0,0,0,3,5,14,2,1,0,2,0,0,0,4,1,7,1,15,0,2,0,0,4,39,0,...,8,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,3,0,0,0,0,0,0,0,0
3,0,0,0,2,0,22,1,12,0,8,0,0,0,12,0,14,0,13,0,4,0,2,2,2,0,4,1,10,0,4,0,0,0,3,0,6,0,19,0,6,0,0,4,16,0,...,0,0,1,0,0,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0
4,0,0,0,35,0,71,0,11,0,3,0,0,0,42,0,2,1,99,0,1,0,0,0,32,0,100,0,34,0,0,0,0,0,94,0,10,4,47,2,1,0,0,12,309,0,...,4,0,0,0,0,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0


In [204]:
joblib.dump(dfvect, 'X_trainLasVegas.joblib', compress=9)

['X_trainLasVegas.joblib']

### Logistic Regression, Checking Final Dataset Outliers

In [3]:
dfvect = joblib.load('X_trainLasVegas.joblib')

In [210]:
X = dfvect
y = trainrowcoords['income']

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=42) #,stratify=y)

In [211]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [212]:
lr.score(X_train, y_train)

1.0

In [213]:
lr.score(X_test, y_test)

0.8632286995515696

In [215]:
lr.coef_.shape

(175, 1482)

In [216]:
joblib.dump(lr, 'LogisticR1basic.joblib', compress=9)

['LogisticR1basic.joblib']

In [ ]:
# landarea, waterarea  # after 1st round models
# transform income, maybe square root

In [5]:
dfvect.describe()

,landarea,waterarea,0_restaurants_near,0_restaurants_far,1_restaurants_near,1_restaurants_far,2_restaurants_near,2_restaurants_far,3_restaurants_near,3_restaurants_far,4_restaurants_near,4_restaurants_far,0_shopping_near,0_shopping_far,1_shopping_near,1_shopping_far,2_shopping_near,2_shopping_far,3_shopping_near,3_shopping_far,4_shopping_near,4_shopping_far,0_food_near,0_food_far,1_food_near,1_food_far,2_food_near,2_food_far,3_food_near,3_food_far,4_food_near,4_food_far,0_beautysvc_near,0_beautysvc_far,1_beautysvc_near,1_beautysvc_far,2_beautysvc_near,2_beautysvc_far,3_beautysvc_near,3_beautysvc_far,4_beautysvc_near,4_beautysvc_far,0_homeservices_near,0_homeservices_far,1_homeservices_near,...,2_hair_extensions_far,3_hair_extensions_near,3_hair_extensions_far,4_hair_extensions_near,4_hair_extensions_far,0_lawyers_near,0_lawyers_far,1_lawyers_near,1_lawyers_far,2_lawyers_near,2_lawyers_far,3_lawyers_near,3_lawyers_far,4_lawyers_near,4_lawyers_far,0_french_near,0_french_far,1_french_near,1_french_far,2_french_near,2_french_far,3_french_near,3_french_far,4_french_near,4_french_far,0_selfstorage_near,0_selfstorage_far,1_selfstorage_near,1_selfstorage_far,2_selfstorage_near,2_selfstorage_far,3_selfstorage_near,3_selfstorage_far,4_selfstorage_near,4_selfstorage_far,0_physicaltherapy_near,0_physicaltherapy_far,1_physicaltherapy_near,1_physicaltherapy_far,2_physicaltherapy_near,2_physicaltherapy_far,3_physicaltherapy_near,3_physicaltherapy_far,4_physicaltherapy_near,4_physicaltherapy_far
count,1784.0,1784.0,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.00000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,...,1784.000000,1784.000000,1784.000000,1784.0,1784.0,1784.000000,1784.000000,1784.0,1784.0,1784.0,1784.0,1784.0,1784.0,1784.0,1784.0,1784.0,1784.0,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.0,1784.0,1784.0,1784.0,1784.0,1784.0,1784.0,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.000000,1784.0,1784.0,1784.0,1784.0
mean,0.0,0.0,2.688341,9.012332,15.566143,53.535314,19.023543,68.424327,3.809417,10.425448,3.777466,7.777466,9.253363,31.596973,3.900785,15.249439,14.627242,56.434978,6.531951,19.039798,3.77130,9.459081,2.062220,6.622758,10.286435,36.274664,8.133408,26.341368,1.679933,4.081839,0.027466,0.126682,5.258969,16.989910,3.940583,13.455717,10.547646,36.427130,2.641816,7.391256,0.179933,0.943386,29.408632,91.663117,0.108184,...,5.326233,0.045404,0.206278,0.0,0.0,0.972534,3.889013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047646,0.224776,0.036435,0.145179,0.075673,0.313341,0.011211,0.072870,1.381166,4.817825,0.000561,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.205157,0.790359,0.007287,0.021300,0.008969,0.039238,0.0,0.0,0.0,0.0
std,0.0,0.0,6.787898,19.427414,22.468925,51.323109,34.603264,94.066663,15.541366,29.990689,20.506058,28.523909,25.348756,49.410826,8.166091,18.169545,38.005439,81.578711,31.274123,73.434723,20.88688,39.800590,7.013331,12.425281,21.307073,43.609786,16.747812,37.048748,7.272218,11.265920,0.224238,0.734293,16.193487,32.479251,8.866736,14.473185,20.047867,57.072243,8.788731,17.957838,0.892808,4.988786,99.826673,171.390297,0.387784,...,17.985757,0.258693,2.968151,0.0,0.0,3.104612,8.644211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.325535,1.356082,0.227931,1.333254,0.629604,1.842333,0.133497,0.628598,7.370600,14.607825,0.023676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.757432,1.506686,0.085076,0.144424,0.094304,0.194214,0.0,0.0,0.0,0.0
min,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [8]:
dfvect.sum().sort_values(ascending=False)

0_homeservices_far         163527
2_restaurants_far          122069
2_shopping_far             100680
1_restaurants_far           95507
0_localservices_far         81369
2_nightlife_far             70271
0_health_far                66675
2_beautysvc_far             64986
1_food_far                  64714
0_eventservices_far         63548
0_auto_far                  63487
0_arts_far                  57661
0_shopping_far              56369
0_homeservices_near         52465
2_food_far                  46993
0_professional_far          46953
2_bars_far                  45385
0_theater_far               39574
2_fashion_far               37674
0_hotelstravel_far          35322
3_shopping_far              33967
2_restaurants_near          33938
0_active_far                32966
0_beautysvc_far             30310
0_physicians_far            30125
0_realestate_far            28825
1_restaurants_near          27770
1_shopping_far              27205
3_nightlife_far             26637
2_eventservice

In [9]:
dfvect.T.sum().sort_values(ascending=False)

361     72177
459     28306
565     26328
1081    14682
609      9066
241      9005
96       8949
54       8939
732      8795
1331     8791
220      8791
47       8765
1441     8734
507      8734
1442     8734
1023     8422
1537     8288
723      8288
578      8276
977      8195
793      8063
1182     8030
1756     8030
217      7961
1322     7961
1318     7961
1320     7961
1319     7961
1317     7961
1316     7961
1315     7961
1321     7961
1757     7936
1183     7936
1583     7906
1582     7906
792      7906
1581     7906
1580     7906
1579     7906
1578     7906
1689     7887
1688     7887
1005     7887
1686     7887
1690     7887
1687     7887
1046     7870
1471     7870
1712     7870
1470     7870
559      7870
711      7860
1718     7835
1071     7835
1743     7795
1149     7795
1744     7795
1291     7737
149      7737
987      7632
1108     7482
1729     7482
1730     7482
1731     7482
1732     7482
1733     7482
1572     7455
781      7455
1573     7455
1022     7447
1245  

In [11]:
outliers = [361,459,565,1081]

In [10]:
trainrowcoords = pd.read_csv('CorpusLasVegasTrainRowCoords.csv',sep=';',compression='gzip')
trainrowcoords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1784 entries, 0 to 1783
Data columns (total 13 columns):
geoid            1784 non-null int64
distance         1784 non-null float64
urban            1784 non-null object
latitude         1784 non-null float64
longitude        1784 non-null float64
blockgroup       1784 non-null int64
income           1784 non-null int64
blockgrouppop    1784 non-null int64
population       1784 non-null int64
landarea         1784 non-null int64
waterarea        1784 non-null int64
block            1784 non-null int64
corpus           1784 non-null object
dtypes: float64(3), int64(8), object(2)
memory usage: 181.3+ KB


In [18]:
trainrowcoords.loc[outliers]

,geoid,distance,urban,latitude,longitude,blockgroup,income,blockgrouppop,population,landarea,waterarea,block,corpus
361,320030067001002,1469.323401,U,36.124488,-115.176362,320030067001,142100,423,21,51558,0,1002,2_bars_far 2_nightlife_far 2_active_far 0_nigh...
459,320030067001020,2376.422301,U,36.116858,-115.166059,320030067001,142100,423,85,42883,0,1020,2_hotels_far 2_hotelstravel_far 2_eventservice...
565,320030067001017,1730.927427,U,36.111572,-115.176464,320030067001,142100,423,26,393718,0,1017,2_food_far 2_burgers_far 2_icecream_far 2_rest...
1081,320030026042001,3085.848569,U,36.107482,-115.161890,320030026042,21591,1586,129,44518,0,2001,0_theater_far 0_arts_far 0_arts_far 0_hotelstr...


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 12882 entries, (36.058, -115.1820861) to (36.183, -115.179353601)
Data columns (total 19 columns):
lat_index.1      12882 non-null float64
long_index.1     12882 non-null float64
name             12882 non-null object
address          12470 non-null object
attributes       11131 non-null object
categories       12851 non-null object
city             12882 non-null object
latitude         12882 non-null float64
longitude        12882 non-null float64
name.1           12882 non-null object
neighborhood     10489 non-null object
postal_code      12882 non-null int64
review_count     12882 non-null int64
stars            12882 non-null float64
state            12882 non-null object
price            12882 non-null int64
main_category    12845 non-null object
distance         12882 non-null float64
alias            12882 non-null object
dtypes: float64(6), int64(3), object(10)
memory usage: 2.0+ MB


In [41]:
df[df['categories'].isnull()]  # not complete info but they still seem ok to keep, in 'other' category

lat_index.1  long_index.1  \
lat_index long_index                               
36.068    -115.153928       36.068   -115.153928   
36.077    -115.153343       36.077   -115.153343   
          -115.153343       36.077   -115.153343   
36.099    -115.175777       36.099   -115.175777   
          -115.175600       36.099   -115.175600   
36.101    -115.140470       36.101   -115.140470   
36.109    -115.210155       36.109   -115.210155   
36.111    -115.198178       36.111   -115.198178   
36.117    -115.157073       36.117   -115.157073   
36.121    -115.169611       36.121   -115.169611   
36.122    -115.189333       36.122   -115.189333   
          -115.169507       36.122   -115.169507   
36.127    -115.203612       36.127   -115.203612   
36.130    -115.122444       36.130   -115.122444   
36.134    -115.177082       36.134   -115.177082   
          -115.176734       36.134   -115.176734   
36.136    -115.230413       36.136   -115.230413   
36.138    -115.242400       36.138   -115.242400   
36.143    -115.245737       36.143   -115.245737   
36.144    -115.136045       36.144   -115.136045   
36.150    -115.255567       36.150   -115.255567   
36.152    -115.209306       36.152   -115.209306   
36.154    -115.243889       36.154   -115.243889   
36.159    -115.227918       36.159   -115.227918   
          -115.194811       36.159   -115.194811   
          -115.134594       36.159   -115.134594   
36.160    -115.184701       36.160   -115.184701   
          -115.149845       36.160   -115.149845   
36.164    -115.168675       36.164   -115.168675   
36.167    -115.146192       36.167   -115.146192   
36.177    -115.169775       36.177   -115.169775   

                                                            name  \
lat_index long_index                                               
36.068    -115.153928                                      Sitel   
36.077    -115.153343              Cheetah Wireless Technologies   
          -115.153343                                 Nomtrition   
36.099    -115.175777                     Vitality International   
          -115.175600                                       24/7   
36.101    -115.140470                        Rancho Vista Realty   
36.109    -115.210155                                 Bella Vita   
36.111    -115.198178                      Casting Entertainment   
36.117    -115.157073                                        Rci   
36.121    -115.169611                                Hotel Sands   
36.122    -115.189333                                Velvet Lion   
          -115.169507                          Cafe' Santa Lucia   
36.127    -115.203612                       All Vegas Properties   
36.130    -115.122444                            Osama Haikal MD   
36.134    -115.177082               Arrow Restoration & Flooring   
          -115.176734                               Jump For Fun   
36.136    -115.230413          Bankers Life and Casualty Company   
36.138    -115.242400                  Image Model Talent Agency   
36.143    -115.245737                         Klumbach and Assoc   
36.144    -115.136045     Atlas Shippers Express Padala & Travel   
36.150    -115.255567                               Plasim Homes   
36.152    -115.209306                              Console Vault   
36.154    -115.243889                         Fortune Properties   
36.159    -115.227918                      Invest America Realty   
          -115.194811                           Autopoint Repair   
          -115.134594                   D & W Auto Sale & Towing   
36.160    -115.184701  Vascular Access Center of Southern Nevada   
          -115.149845            Nevada Professional Collections   
36.164    -115.168675                         Martin Gloria L MD   
36.167    -115.146192                             Carson St Cafe   
36.177    -115.169775                          Boulder Placement   

                                                address  \
la

In [43]:
# calculates number of sub-categories each business belongs to, to try to find cause of lat/longs
# with much higher total category counts
df['alias'].map(lambda x : len(x.split(' '))).sort_values(ascending=False)
# no abnormally high category counts, will keep all businesses

lat_index  long_index 
36.129     -115.171130    14
36.143     -115.262227    13
36.107     -115.181590    12
36.116     -115.198279    12
36.084     -115.192942    12
36.069     -115.177945    12
36.118     -115.168662    11
36.145     -115.213750    11
36.077     -115.243196    11
36.124     -115.171518    11
36.141     -115.142648    11
36.115     -115.206794    11
36.160     -115.222210    10
36.070     -115.173856    10
36.115     -115.207537    10
36.123     -115.170194    10
36.129     -115.142491    10
36.069     -115.191228    10
36.118     -115.170430    10
36.093     -115.175868    10
36.136     -115.230413    10
36.158     -115.137781    10
36.167     -115.154643    10
36.159     -115.202586    10
36.126     -115.168252    10
36.099     -115.167874    10
36.115     -115.204294    10
36.102     -115.156860    10
36.168     -115.140640    10
36.143     -115.221700    10
36.090     -115.243195     9
36.119     -115.175200     9
36.122     -115.174000     9
36.141     -115.1367